In [1]:
import pandas as pd
import psycopg2
import csv
import sys
import datasets
import get_notes
from concat_and_split_full import concat_data
from collections import Counter, defaultdict
import operator
import numpy as np
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv('./full_events.csv')
len(df['HADM_ID'].unique())

58361

In [4]:
df = df.drop(df[(df['HADM_ID'].isnull()) | (df['TEXT'].isnull())].index)

In [5]:
df = df.sort_values(['SUBJECT_ID', 'HADM_ID', 'CHARTDATE', 'CHARTTIME'])
dflabels = pd.read_csv('./DiagICD.csv')
dflabels = dflabels.sort_values(['SUBJECT_ID', 'HADM_ID'])

In [6]:
len(df['HADM_ID'].unique()), len(dflabels['HADM_ID'].unique())

(58361, 58976)

In [7]:

hadm_ids = set(df['HADM_ID'])
with open('./DiagICD.csv', 'r') as lf:
    with open('./DiagICD_filtered.csv', 'w') as of:
        w = csv.writer(of)
        w.writerow(['SUBJECT_ID', 'HADM_ID', 'ICD9_CODE', 'ADMITTIME', 'DISCHTIME'])
        r = csv.reader(lf)
        #header
        next(r)
        for i,row in enumerate(r):
            hadm_id = int(row[2])
            #print(hadm_id)
            #break
            if hadm_id in hadm_ids:
                w.writerow(row[1:3] + [row[-1], '', ''])

In [8]:
dflabels = pd.read_csv('./DiagICD_filtered.csv', index_col=None)

In [9]:
len(dflabels['HADM_ID'].unique()), len(df['HADM_ID'].unique())

(58361, 58361)

In [10]:
dflabels = dflabels.sort_values(['SUBJECT_ID', 'HADM_ID'])
dflabels.to_csv('./DiagICD_filtered.csv', index=False)

In [ ]:
sorted_file = './full_events.csv'
df.to_csv(sorted_file, index=False)

In [ ]:
labeled = concat_data('./DiagICD_filtered.csv', sorted_file)

In [2]:
labeled = pd.read_csv('./notes_labeled_full.csv', index_col=None)

In [3]:
labeled

,SUBJECT_ID,HADM_ID,TEXT,LABELS
0,2,163353,note nursing other neonatology attending triag...,V30.01;V05.3;V29.0
1,2,163353,note nursing other nursing transfer note pt ad...,V30.01;V05.3;V29.0
2,3,145834,note radiology pm ct abdomen w o contrast ct p...,038.9;785.59;584.9;427.5;410.71;428.0;682.6;42...
3,3,145834,note radiology pm chest portable ap clip clip ...,038.9;785.59;584.9;427.5;410.71;428.0;682.6;42...
4,3,145834,note radiology pm chest portable ap clip clip ...,038.9;785.59;584.9;427.5;410.71;428.0;682.6;42...
...,...,...,...,...
1851339,99999,113369,note physician sicu hpi f pod1 s p plif with r...,756.12;786.1;401.9;250.00;413.9
1851340,99999,113369,note physician sicu hpi f pod1 s p plif with r...,756.12;786.1;401.9;250.00;413.9
1851341,99999,113369,note nursing yo f with history of neurogenic c...,756.12;786.1;401.9;250.00;413.9
1851342,99999,113369,note nursing yo f with history of neurogenic c...,756.12;786.1;401.9;250.00;413.9


In [4]:
labeled_300 = labeled.drop(labeled[labeled.TEXT.str.len()<300].index)

In [5]:
labeled_300

,SUBJECT_ID,HADM_ID,TEXT,LABELS
0,2,163353,note nursing other neonatology attending triag...,V30.01;V05.3;V29.0
1,2,163353,note nursing other nursing transfer note pt ad...,V30.01;V05.3;V29.0
2,3,145834,note radiology pm ct abdomen w o contrast ct p...,038.9;785.59;584.9;427.5;410.71;428.0;682.6;42...
3,3,145834,note radiology pm chest portable ap clip clip ...,038.9;785.59;584.9;427.5;410.71;428.0;682.6;42...
4,3,145834,note radiology pm chest portable ap clip clip ...,038.9;785.59;584.9;427.5;410.71;428.0;682.6;42...
...,...,...,...,...
1851339,99999,113369,note physician sicu hpi f pod1 s p plif with r...,756.12;786.1;401.9;250.00;413.9
1851340,99999,113369,note physician sicu hpi f pod1 s p plif with r...,756.12;786.1;401.9;250.00;413.9
1851341,99999,113369,note nursing yo f with history of neurogenic c...,756.12;786.1;401.9;250.00;413.9
1851342,99999,113369,note nursing yo f with history of neurogenic c...,756.12;786.1;401.9;250.00;413.9


In [13]:
Y=50
codes_50 = ['401.9',
 '428.0',
 '427.31',
 '414.01',
 '584.9',
 '250.00',
 '272.4',
 '518.81',
 '599.0',
 '530.81',
 '272.0',
 'V05.3',
 'V29.0',
 '285.9',
 '244.9',
 '486',
 '285.1',
 '276.2',
 '496',
 '995.92',
 'V58.61',
 '038.9',
 '507.0',
 'V30.00',
 '585.9',
 '311',
 '403.90',
 '305.1',
 '412',
 '410.71',
 '287.5',
 'V45.81',
 '276.1',
 '424.0',
 'V15.82',
 '511.9',
 'V45.82',
 'V30.01',
 '403.91',
 '785.52',
 '424.1',
 'V58.67',
 '427.89',
 '327.23',
 '997.1',
 '584.5',
 '276.0',
 '774.2',
 '493.90',
 '518.0']

In [14]:
csv.field_size_limit(sys.maxsize)
for splt in ['train', 'dev', 'test']:
    print(splt)
    hadm_ids = set()
    with open('./hadm_ids/%s_50_hadm_ids.csv' %splt, 'r') as f:
        for line in f:
            hadm_ids.add(line.rstrip())
    with open('./notes_labeled_full.csv', 'r') as f:
        with open('./%s_%s.csv' % (splt, str(Y)), 'w') as of:
            r = csv.reader(f)
            w = csv.writer(of)
            #header
            w.writerow(next(r))
            i = 0
            for row in r:
                hadm_id = row[1]
                if hadm_id not in hadm_ids:
                    continue
                codes = set(str(row[3]).split(';'))
                filtered_codes = codes.intersection(set(codes_50))
                if len(filtered_codes) > 0:
                    w.writerow(row[:3] + [';'.join(filtered_codes)])
                    i += 1

train
dev
test


In [15]:
for splt in ['train', 'dev', 'test']:
    filename = './%s_%s.csv' % (splt, str(Y))
    df = pd.read_csv(filename)
    df['length'] = df.apply(lambda row: len(str(row['TEXT']).split()), axis=1)
    df = df.sort_values(['length'])
    df.to_csv('./%s_%s.csv' % (splt, str(Y)), index=False)

In [16]:
train_50 = pd.read_csv('./train_50.csv', index_col=None)

In [17]:
train_50

,SUBJECT_ID,HADM_ID,TEXT,LABELS,length
0,67906,122154,title,427.31;038.9;584.5;995.92;285.1;518.81;287.5;2...,1
1,22010,146210,title,428.0;285.1;V58.67;424.0,1
2,72671,188200,title,427.31;276.2;311;486;518.81,1
3,78481,143054,events,038.9;584.5;285.1;518.81;424.0;401.9,1
4,31209,106125,title,276.2;412;414.01;327.23;V58.61;272.0,1
...,...,...,...,...,...
276541,57321,171040,admission date discharge date date of birth se...,250.00;427.31;276.2;427.89;428.0;584.5;518.81;...,6081
276542,80342,160825,admission date discharge date date of birth se...,276.2;584.9;518.81;038.9,6120
276543,65176,173812,admission date discharge date date of birth se...,038.9;584.5;995.92;285.1;518.81;287.5;511.9,6487
276544,51821,197028,admission date discharge date date of birth se...,486;584.5;518.81;287.5;276.0;493.90;511.9,6787


In [ ]:
train_50 = pd.read_csv('./train_50.csv', index_col=None)
dev_50 = pd.read_csv('./dev_50.csv', index_col=None)
test_50 = pd.read_csv('./test_50.csv', index_col=None)

In [ ]:
full_df = pd.concat([train_50, dev_50, test_50], ignore_index=True)

In [ ]:
# split labels by ";", then convert to list
def split_lab (x):
    #print(x)
    return x.split(";")

full_df['LABELS'] = full_df['LABELS'].apply(split_lab)
#full_df['TEXT'] = full_df['TEXT'].apply(split_lab)

full_df.head()

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer(sparse_output=True)

In [ ]:
#change label to one-hot encoding per code
full_df = full_df.join(
            pd.DataFrame.sparse.from_spmatrix(
                mlb.fit_transform(full_df.pop('LABELS')),
                columns=mlb.classes_))

full_df.head()

In [ ]:
icd_classes_50 = mlb.classes_
print(icd_classes_50)

In [ ]:
# Convert columns to list of one hot encoding
icd_classes_50 = mlb.classes_

full_df['labels'] = full_df[icd_classes_50].values.tolist()
#train_df.sort_values(['length'], ascending=False, inplace=True)
full_df.head()

In [ ]:
full_df.to_csv('./notes_full_label.csv', index=False)

In [ ]:
Y=50
csv.field_size_limit(sys.maxsize)
for splt in ['train', 'dev', 'test']:
    print(splt)
    hadm_ids = set()
    with open('./hadm_ids/%s_50_hadm_ids.csv' %splt, 'r') as f:
        for line in f:
            hadm_ids.add(line.rstrip())
    with open('./notes_full_label.csv', 'r') as f:
        with open('./%s_%s.csv' % (splt, str(Y)), 'w') as of:
            r = csv.reader(f)
            w = csv.writer(of)
            #header
            w.writerow(next(r))
            i = 0
            for row in r:
                hadm_id = row[1]
                if hadm_id not in hadm_ids:
                    continue

                w.writerow(row)
                i += 1

In [ ]:
for splt in ['train', 'dev', 'test']:
    filename = './%s_%s.csv' % (splt, str(Y))
    df = pd.read_csv(filename)

    df = df.sort_values(['length'])
    df.to_csv('./%s_%s.csv' % (splt, str(Y)), index=False)

In [ ]:
train_50 = pd.read_csv('./train_50.csv', index_col=None)
train_50

In [ ]:
sns.kdeplot(dev_50.length[dev_50['length']<200])

In [ ]:
dev_50 = pd.read_csv('./dev_50.csv', index_col=None)
dev_50

In [18]:
Y=10
codes_10 = ['401.9',
 '428.0',
 '427.31',
 '414.01',
 '584.9',
 '250.00',
 '272.4',
 '518.81',
 '599.0',
 '530.81']

In [19]:
csv.field_size_limit(sys.maxsize)
for splt in ['train', 'dev', 'test']:
    print(splt)
    hadm_ids = set()
    with open('./hadm_ids/%s_50_hadm_ids.csv' %splt, 'r') as f:
        for line in f:
            hadm_ids.add(line.rstrip())
    with open('./notes_labeled_full.csv', 'r') as f:
        with open('./%s_%s.csv' % (splt, str(Y)), 'w') as of:
            r = csv.reader(f)
            w = csv.writer(of)
            #header
            w.writerow(next(r))
            i = 0
            for row in r:
                hadm_id = row[1]
                if hadm_id not in hadm_ids:
                    continue
                codes = set(str(row[3]).split(';'))
                filtered_codes = codes.intersection(set(codes_10))
                if len(filtered_codes) > 0:
                    w.writerow(row[:3] + [';'.join(filtered_codes)])
                    i += 1

train
dev
test


In [20]:
for splt in ['train', 'dev', 'test']:
    filename = './%s_%s.csv' % (splt, str(Y))
    df = pd.read_csv(filename)
    df['length'] = df.apply(lambda row: len(str(row['TEXT']).split()), axis=1)
    df = df.sort_values(['length'])
    df.to_csv('./%s_%s.csv' % (splt, str(Y)), index=False)

In [ ]:
train_10 = pd.read_csv('./train_10.csv', index_col=None)
dev_10 = pd.read_csv('./dev_10.csv', index_col=None)
test_10 = pd.read_csv('./test_10.csv', index_col=None)

In [ ]:
full_df = pd.concat([train_10, dev_10, test_10], ignore_index=True)

In [ ]:
train_10

In [ ]:
# split labels by ";", then convert to list
def split_lab (x):
    #print(x)
    return x.split(";")

full_df['LABELS'] = full_df['LABELS'].apply(split_lab)
#full_df['TEXT'] = full_df['TEXT'].apply(split_lab)

full_df.head()

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer(sparse_output=True)

In [ ]:
#change label to one-hot encoding per code
full_df = full_df.join(
            pd.DataFrame.sparse.from_spmatrix(
                mlb.fit_transform(full_df.pop('LABELS')),
                columns=mlb.classes_))

full_df.head()

In [ ]:
# Convert columns to list of one hot encoding
icd_classes_50 = mlb.classes_

full_df['labels'] = full_df[icd_classes_50].values.tolist()
#train_df.sort_values(['length'], ascending=False, inplace=True)
full_df

In [ ]:
full_df.to_csv('./notes_full_label_10.csv', index=False)

In [ ]:
csv.field_size_limit(sys.maxsize)
for splt in ['train', 'dev', 'test']:
    print(splt)
    hadm_ids = set()
    with open('./hadm_ids/%s_50_hadm_ids.csv' %splt, 'r') as f:
        for line in f:
            hadm_ids.add(line.rstrip())
    with open('./notes_full_label_10.csv', 'r') as f:
        with open('./%s_%s.csv' % (splt, str(Y)), 'w') as of:
            r = csv.reader(f)
            w = csv.writer(of)
            #header
            w.writerow(next(r))
            i = 0
            for row in r:
                hadm_id = row[1]
                if hadm_id not in hadm_ids:
                    continue

                w.writerow(row)
                i += 1

In [ ]:
for splt in ['train', 'dev', 'test']:
    filename = './%s_%s.csv' % (splt, str(Y))
    df = pd.read_csv(filename)

    df = df.sort_values(['length'])
    df.to_csv('./%s_%s.csv' % (splt, str(Y)), index=False)

In [ ]:
train_10 = pd.read_csv('./train_10.csv', index_col=None)
train_10